# Hiscores Ironman Account Type Checker and Organized API Pulls
## Loops through entire clan, saves their stats @ timestamps

In [25]:
#Package Imports
import requests
import json
import pandas as pd
import numpy as np
from datetime import date

In [26]:
#Clan List pulled via Clanmate export runelite plugin
#For this demo I've just selected three random players I saw while mining
ClanList = pd.read_csv('members.csv')

In [27]:
#Checking out the dataframe
#ClanList 
ClanName = ClanList['RSN'].to_list()

In [28]:
#checking out list of random names I picked
ClanName

['N54 Gang', 'SoulfulDink', 'KingJacula']

In [29]:
#Checking current size of clan as of most recent claname export
ClanSize = len(ClanName)
ClanSize

#Note - Clansmate RSN's are strings with spaces allowed

3

In [30]:
#Standard Imports
#General commments, this is a fairly inneficient function, chose to hack it together instead of going foor a list 
#remmoval process which I'll add for V2
def AccountStatsPlusType(RSN):

    #import requests
    #import json
    #import pandas as pd            #commented out as it is not best practice to have imports inside a function
    #import numpy as np
    #from datetime import date

    SkillsOrder= ['Overall', 'Attack', 'Defence', 'Strength', 'Hitpoints', 'Ranged', 'Prayer', 'Magic', 'Cooking', 'Woodcutting', 'Fletching', 'Fishing', 'Firemaking', 'Crafting', 'Smithing', 'Mining', 'Herblore', 'Agility', 'Thieving', 'Slayer', 'Farming', 'Runecraft', 'Hunter', 'Construction', 'empty',   'Bounty Hunter - Hunter',    'Bounty Hunter - Rogue',    'Clue Scrolls (all)',    'Clue Scrolls (beginner)',    'Clue Scrolls (easy)',    'Clue Scrolls (medium)',    'Clue Scrolls (hard)',    'Clue Scrolls (elite)',    'Clue Scrolls (master)',    'LMS - Rank',    'PvP Arena - Rank',    'Soul Wars Zeal',    'Rifts closed',    'Abyssal Sire',    'Alchemical Hydra',    'Artio',    'Barrows Chests',    'Bryophyta',    'Callisto',    "Cal'varion",    'Cerberus',    'Chambers of Xeric',    'Chambers of Xeric: Challenge Mode',    'Chaos Elemental',    'Chaos Fanatic',    'Commander Zilyana',    'Corporeal Beast',    'Crazy Archaeologist',    'Dagannoth Prime',    'Dagannoth Rex',    'Dagannoth Supreme',    'Deranged Archaeologist',    'General Graardor',    'Giant Mole',    'Grotesque Guardians',    'Hespori',    'Kalphite Queen',    'King Black Dragon',    'Kraken',    "Kree'Arra",    "K'ril Tsutsaroth",    'Mimic',    'Nex',    'Nightmare',    "Phosani's Nightmare",    'Obor',    'Phantom Muspah',    'Sarachnis',    'Scorpia',    'Skotizo',    'Spindel',    'Tempoross',    'The Gauntlet',    'The Corrupted Gauntlet',    'Theatre of Blood',    'Theatre of Blood: Hard Mode',    'Thermonuclear Smoke Devil',    'Tombs of Amascut',    'Tombs of Amascut: Expert Mode',    'TzKal-Zuk',    'TzTok-Jad',    'Venenatis',    "Vet'ion",    'Vorkath',    'Wintertodt',    'Zalcano',    'Zulrah']

    OverallHS = f"https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player={RSN}"
    APIhit = f"https://secure.runescape.com/m=hiscore_oldschool_ironman/index_lite.ws?player={RSN}"
    HC_APIhit = f"https://secure.runescape.com/m=hiscore_oldschool_hardcore_ironman/index_lite.ws?player={RSN}"
    UIM_APIhit = f"https://secure.runescape.com/m=hiscore_oldschool_ultimate/index_lite.ws?player={RSN}"

    #connnect to the IM API
    IMresponse = requests.get(APIhit)
    IMdata = IMresponse.text

    #connect to the HC API
    HCresponse = requests.get(HC_APIhit)
    HCdata = HCresponse.text   

    #connect to the HC API
    UIMresponse = requests.get(UIM_APIhit)
    UIMdata = UIMresponse.text   

    #connect to normal API
    GIMresponse = requests.get(OverallHS)
    GIMdata = GIMresponse.text      

    # replace all instances of '\n' with a comma
    IM_Sub = IMdata.splitlines()
    HC_Sub = HCdata.splitlines()
    UIM_Sub = UIMdata.splitlines()
    GIM_Sub = GIMdata.splitlines()
    df = pd.DataFrame(SkillsOrder)

    #ETL

    #Create empty dataframe
    GIM = pd.DataFrame()
    IM = pd.DataFrame()
    UIM = pd.DataFrame()
    HCIM = pd.DataFrame()
    
    
    
    GIM['API'] = GIM_Sub[0:24]
    GIM[['Rank','Level','Exp']] = GIM['API'].str.split(',',expand=True)
    #Rename first column appropriately
    GIM.columns = ['Skill', *GIM.columns[1:]]
    GIM['Skill'] = SkillsOrder[0:24]
    #Add timestamp for daily delta calculations
    GIM['Date'] = datetime.now()
    GIM['RSN'] = RSN
    GIM['Player Type'] = 'Main/GIM/UGIM'
    GIM['IM_Order'] = 4  
    
   
    try:
        IM['API'] = IM_Sub[0:24]
        IM[['Rank','Level','Exp']] = IM['API'].str.split(',',expand=True)
        #Rename first column appropriately
        IM.columns = ['Skill', *IM.columns[1:]]
        IM['Skill'] = SkillsOrder[0:24]
        #Add timestamp for daily delta calculations
        IM['Date'] = datetime.now()
        IM['RSN'] = RSN
        IM['Player Type'] = 'IM'
        IM['IM_Order'] = 3   
    except:
        pass
    try:
        HCIM['API'] = HC_Sub[0:24]
        HCIM[['Rank','Level','Exp']] = HCIM['API'].str.split(',',expand=True)
        #Rename first column appropriately
        HCIM.columns = ['Skill', *HCIM.columns[1:]]
        HCIM['Skill'] = SkillsOrder[0:24]
        #Add timestamp for daily delta calculations
        HCIM['Date'] = datetime.now()
        HCIM['RSN'] = RSN
        HCIM['Player Type'] = 'HCIM'
        HCIM['IM_Order'] = 2  
    except:
        pass
    try:
        UIM['API'] = UIM_Sub[0:24]
        UIM[['Rank','Level','Exp']] = UIM['API'].str.split(',',expand=True)
        #Rename first column appropriately
        UIM.columns = ['Skill', *UIM.columns[1:]]
        UIM['Skill'] = SkillsOrder[0:24]
        #Add timestamp for daily delta calculations
        UIM['Date'] = datetime.now()
        UIM['RSN'] = RSN
        UIM['Player Type'] = 'UIM'
        UIM['IM_Order'] = 1 
    except:
        pass
    
    lst = [GIM,IM,UIM,HCIM]
    df = pd.concat(lst)
    df['Exp'] = pd.to_numeric(df['Exp'])
    df['IM_Order'] = pd.to_numeric(df['IM_Order'])
    df = df.sort_values(by='IM_Order', ascending=False)
    df = df.dropna(thresh=5)
    
    typing = df[df['Skill'] == 'Overall']
    typing = typing.query('Exp == Exp.max()')
    
    df = df[df['Player Type'] == typing.iloc[-1,6]]
    
        
        
    return df

In [ ]:
## Looping portion of clan
# For reference, a clan size of approximately 350 will take ~7 minutes to complete using 16GB ram

In [31]:
dfs = []
for RSN in ClanName:
    print(RSN)
    try:
        ClanName = AccountStatsPlusType(RSN)
        dfs.append(ClanName)
        ClanHiScoresDs = pd.concat(dfs, ignore_index=True)
    except:
        continue

N54 Gang
SoulfulDink
KingJacula


In [32]:
#comment this part out after first run through
#ClanHiScoresDs.to_csv('ClanHiscoresDataset.csv')

- The job can be ran on a timer via a cron job or a glue job on AWS
- AWS offers some nice dashboarding capabilityies built in via quicksite

- From here a master dataframe is created and stored in AWS (or locally) it is appended to itself in a method similar to above every time the script is ran.

In [34]:
#The database size can grow rapidlty based on the number of members of the clan and refresh frequency.
PreviousHS = pd.read_csv('ClanHiscoresDataset.csv')
MostRecentUpdate = ClanHiScoresDs
MostRecentUpdate = pd.concat([PreviousHS,MostRecentUpdate], ignore_index=True)
MostRecentUpdate

,Unnamed: 0,Skill,Rank,Level,Exp,Date,RSN,Player Type,IM_Order,API
0,0.0,Overall,580850,1766,55636203.0,2023-05-22,N54 Gang,Main/GIM/UGIM,4.0,NaN
1,1.0,Attack,605106,86,3910407.0,2023-05-22,N54 Gang,Main/GIM/UGIM,4.0,NaN
2,2.0,Hunter,754263,69,703272.0,2023-05-22,N54 Gang,Main/GIM/UGIM,4.0,NaN
3,3.0,Runecraft,517176,64,448512.0,2023-05-22,N54 Gang,Main/GIM/UGIM,4.0,NaN
4,4.0,Farming,717908,72,947654.0,2023-05-22,N54 Gang,Main/GIM/UGIM,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...
139,NaN,Ranged,14634,99,73819813.0,2023-05-22,KingJacula,Main/GIM/UGIM,4.0,NaN
140,NaN,Hitpoints,12480,99,73715284.0,2023-05-22,KingJacula,Main/GIM/UGIM,4.0,NaN
141,NaN,Strength,9208,99,70675816.0,2023-05-22,KingJacula,Main/GIM/UGIM,4.0,NaN
142,NaN,Defence,11061,99,28519871.0,2023-05-22,KingJacula,Main/GIM/UGIM,4.0,NaN
